In [ ]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# %matplotlib inline
# from matplotlib.pyplot import rcParams
rcParams['figure.figsize'] = 10,6
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.optimizers import Adam
import os

In [ ]:
data_path = "/content/drive/MyDrive/Splitted"

In [ ]:
train_dir = "/content/drive/MyDrive/Splitted/train"
test_dir = "/content/drive/MyDrive/Splitted/test"

In [ ]:
for i in os.listdir(train_dir):
  print(i, "--->", len(os.listdir(os.path.join(train_dir, i))))

Cheetah ---> 80
Fire ---> 80
Hippopotamus ---> 76
Human ---> 80
Monkey ---> 80
Spinny babbler ---> 80
Spotted Dove ---> 80
Wolf ---> 80
bear ---> 80
deer ---> 80
elephant ---> 80
rhinoceros ---> 80
tiger ---> 80
forest ---> 78


In [ ]:
for i in os.listdir(test_dir):
  print(i, "--->", len(os.listdir(os.path.join(test_dir, i))))
  

Cheetah ---> 22
Fire ---> 14
Hippopotamus ---> 19
Human ---> 20
Monkey ---> 20
Spinny babbler ---> 24
Spotted Dove ---> 24
Wolf ---> 25
bear ---> 20
deer ---> 22
elephant ---> 20
rhinoceros ---> 23
tiger ---> 29
forest ---> 20


In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
)
train_generator = train_datagen.flow_from_directory(
    directory = "/content/drive/MyDrive/Splitted/train",
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 1112 images belonging to 14 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = test_datagen.flow_from_directory(
    directory = "/content/drive/MyDrive/Splitted/test",
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 302 images belonging to 14 classes.


In [ ]:
pretrained_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in pretrained_model.layers:
  layer.trainable = False

inputs = layers.Input(shape = (224,224,3),name = "input_layer")
x = pretrained_model(inputs)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(32, activation = 'relu')(x)
x = layers.Dense(14, activation = 'softmax')(x)
model = Model(inputs, x)
model.summary()

87910968/87910968 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 32)                65568     
                                                                 
 dense_1 (Dense)             (None, 14)                462       
                                                                 
Total params: 21,868,814
Trainable params: 66,030
Non-trainabl

In [ ]:

lr = 0.0001
optimizer = Adam(lr=lr)
model.compile(optimizer=optimizer,
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 32)                65568     
                                                                 
 dense_1 (Dense)             (None, 14)                462       
                                                                 
Total params: 21,868,814
Trainable params: 66,030
Non-trainable params: 21,802,784
____________________________________________

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

history = model.fit(
            train_generator,
            steps_per_epoch = 35,
            epochs=3,
            validation_data=validation_generator,
            validation_steps=200,
            verbose = 1
            )

Epoch 1/3
35/35 [==============================] - ETA: 0s - loss: 2.4520 - accuracy: 0.2266

35/35 [==============================] - 382s 11s/step - loss: 2.4520 - accuracy: 0.2266 - val_loss: 2.1263 - val_accuracy: 0.3212
Epoch 2/3
35/35 [==============================] - 21s 609ms/step - loss: 1.7512 - accuracy: 0.5162
Epoch 3/3
35/35 [==============================] - 23s 651ms/step - loss: 1.1962 - accuracy: 0.7401


In [ ]:
model.save("inception.h5")

In [ ]:
loss, accuracy  = model.evaluate(validation_generator)
print("Test accuracy: ", accuracy)

10/10 [==============================] - 10s 889ms/step - loss: 1.0935 - accuracy: 0.7781
Test accuracy:  0.7781456708908081
